In [62]:
import argparse
import os
import shutil

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from datasets_prep.dataset import create_dataset
from diffusion import sample_from_model, sample_posterior, \
    q_sample_pairs, get_time_schedule, \
    Posterior_Coefficie
nts, Diffusion_Coefficients
#from DWT_IDWT.DWT_IDWT_layer import DWT_2D, IDWT_2D
#from pytorch_wavelets import DWTForward, DWTInverse
from torch.multiprocessing import Process
from utils import init_processes, copy_source, broadcast_params
import yaml

from ldm.util import instantiate_from_config
from omegaconf import OmegaConf

ImportError: cannot import name 'Posterior_Coefficie' from 'diffusion' (/mnt/box/MyFolder/project/Latent-DDGAN/diffusion.py)

In [63]:
def load_model_from_config(config_path, ckpt):
    print(f"Loading model from {ckpt}")
    config = OmegaConf.load(config_path)
    pl_sd = torch.load(ckpt, map_location="cpu")
    #global_step = pl_sd["global_step"]
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    model = model.first_stage_model
    model.cuda()
    model.eval()
    del m
    del u
    del pl_sd
    return model

def grad_penalty_call(args, D_real, x_t):
    grad_real = torch.autograd.grad(
        outputs=D_real.sum(), inputs=x_t, create_graph=True
    )[0]
    grad_penalty = (
        grad_real.view(grad_real.size(0), -1).norm(2, dim=1) ** 2
    ).mean()

    grad_penalty = args.r1_gamma / 2 * grad_penalty
    grad_penalty.backward()

In [65]:

from ldm.util import instantiate_from_config
config_path = "./autoencoder/config/cifar10_16x16x4.yaml"
ckpt_path = "./autoencoder/weight/16x16x4_551.ckpt"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

AutoEncoder = instantiate_from_config(config['model'])


checkpoint = torch.load(ckpt_path, map_location=device)
AutoEncoder.load_state_dict(checkpoint['state_dict'])
AutoEncoder.eval()
AutoEncoder.to(device)

making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
Working with z of shape (1, 4, 16, 16) = 1024 dimensions.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.2, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (no

In [67]:
config_path = "./autoencoder/config/COCO_config.yaml"
ckpt_path = "./autoencoder/weight/16x16x4_551.ckpt"

AutoEncoder = load_model_from_config(config_path, ckpt_path)

Loading model from ./autoencoder/weight/16x16x4_551.ckpt


ScannerError: mapping values are not allowed here
  in "/mnt/box/MyFolder/project/Latent-DDGAN/autoencoder/config/COCO_config.yaml", line 56, column 17

In [53]:
import os

import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.datasets import CIFAR10, STL10

from datasets_prep.coco import CustomCocoCaptions

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.transforms.CenterCrop(256),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset = CustomCocoCaptions(
    name="train2017",
    root=os.path.join("./data/coco", "train2017"),
    annFile=os.path.join(
    "./data/coco", 'annotations', 'captions_train2017.json'), 
    transform=train_transform)

data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,           
                                         )

loading annotations into memory...
Done (t=0.69s)
creating index...
index created!


In [44]:
dataset = CIFAR10("./data/cifar-10", train=True, transform=transforms.Compose([
            transforms.Resize(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]), download=True)
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,           
                                         )

Files already downloaded and verified


In [54]:
tmp = iter(data_loader)
x, y = next(tmp)

In [55]:
device = "cuda"
scale_factor = 6.0
x0 = x.to(device, non_blocking=True)
with torch.no_grad():
    posterior = AutoEncoder.encode(x0)
    real_data = posterior.sample().detach()
#print("MIN:{}, MAX:{}".format(real_data.min(), real_data.max()))
real_data = real_data / scale_factor #300.0  # [-1, 1]

In [56]:
real_data *= scale_factor #300
with torch.no_grad():
    real_data = AutoEncoder.decode(real_data)

real_data = (torch.clamp(real_data, -1, 1) + 1) / 2  # 0-1 

torchvision.utils.save_image(
    real_data, os.path.join("./", 'real_data.png'))

OUTTTT


In [69]:
!bash run.sh coco_256 train 1

MASTER_PORT=6038
The work dir is:  /mnt/box/MyFolder/project/Latent-DDGAN
coco_256 train 1
==> Training LDDGAN
starting in debug mode
git root error: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-parse --show-toplevel
  stderr: 'fatal: detected dubious ownership in repository at '/mnt/box/MyFolder/project/Latent-DDGAN'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/box/MyFolder/project/Latent-DDGAN'
git root error: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-parse --show-toplevel
  stderr: 'fatal: detected dubious ownership in repository at '/mnt/box/MyFolder/project/Latent-DDGAN'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/box/MyFolder/project/Latent-DDGAN'
wandb: Currently logged in as: kotomiya07. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracki